# Лабораторная работа №2

## Задание 1
>Провести классификацию найденного датасета, методом машины опорных векторов.

Импорт библиотек

In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.svm import SVC

Загрузка датасета

In [26]:
df = pd.read_csv('metal.csv', encoding='utf-8')
df['Mode']= df['Mode'].map({1: True, 0: False}).astype(bool)
print(df.head(3))
print(df.dtypes)

               Spotify ID              Artist IDs Track Name  \
0  19vHgVS1aukRiQWhTqfKnE  1Ffb6ejR6Fe5IamqA5oRUF   DArkSide   
1  5BgnL6gHauuvxe4Ok6W1aC  1Ffb6ejR6Fe5IamqA5oRUF       LosT   
2  0M3adYbGtyRHACP86dey1H  1Ffb6ejR6Fe5IamqA5oRUF     Throne   

          Album Name        Artist Name(s) Release Date  Duration (ms)  \
0           DArkSide  Bring Me The Horizon   10/18/2023         164790   
1               LosT  Bring Me The Horizon     5/4/2023         205368   
2  That's The Spirit  Bring Me The Horizon    9/11/2015         191520   

   Popularity                                Added By              Added At  \
0          74  spotify:user:woth5clwe5p856iw52alp7uek  2023-10-25T13:10:28Z   
1          72  spotify:user:woth5clwe5p856iw52alp7uek  2023-10-25T13:10:28Z   
2          76  spotify:user:woth5clwe5p856iw52alp7uek  2023-10-25T13:10:28Z   

   ... Key  Loudness   Mode  Speechiness  Acousticness  Instrumentalness  \
0  ...   7    -3.018   True       0.1460      0.00035

Удаление ненужных столбцов из датасета

In [27]:
df = df.drop(['Spotify ID', 'Added By', 'Added At', 'Time Signature'], axis=1)
print(df.head(3))

               Artist IDs Track Name         Album Name        Artist Name(s)  \
0  1Ffb6ejR6Fe5IamqA5oRUF   DArkSide           DArkSide  Bring Me The Horizon   
1  1Ffb6ejR6Fe5IamqA5oRUF       LosT               LosT  Bring Me The Horizon   
2  1Ffb6ejR6Fe5IamqA5oRUF     Throne  That's The Spirit  Bring Me The Horizon   

  Release Date  Duration (ms)  Popularity  \
0   10/18/2023         164790          74   
1     5/4/2023         205368          72   
2    9/11/2015         191520          76   

                                              Genres  Danceability  Energy  \
0  melodic metalcore,metalcore,modern rock,rock,u...         0.387   0.940   
1  melodic metalcore,metalcore,modern rock,rock,u...         0.396   0.939   
2  melodic metalcore,metalcore,modern rock,rock,u...         0.426   0.888   

   Key  Loudness   Mode  Speechiness  Acousticness  Instrumentalness  \
0    7    -3.018   True       0.1460      0.000352          0.000003   
1    9    -3.155  False       0.2270 

Удаление строк с пропущенными значениями

In [28]:
df.dropna(inplace=True)

Предобработка данных

In [29]:
# Кодирование категориальных признаков
label_encoders = {}
for column in df.select_dtypes(include=['object']).columns:
    label_encoders[column] = LabelEncoder()
    df[column] = label_encoders[column].fit_transform(df[column])
print(df.head(3))
print(df.dtypes)

   Artist IDs  Track Name  Album Name  Artist Name(s)  Release Date  \
0         212        2152         631             182           206   
1         212        5399        1647             182          1178   
2         212        9575        2696             182          1740   

   Duration (ms)  Popularity  Genres  Danceability  Energy  Key  Loudness  \
0         164790          74     972         0.387   0.940    7    -3.018   
1         205368          72     972         0.396   0.939    9    -3.155   
2         191520          76     972         0.426   0.888    0    -3.720   

    Mode  Speechiness  Acousticness  Instrumentalness  Liveness  Valence  \
0   True       0.1460      0.000352          0.000003     0.628    0.456   
1  False       0.2270      0.000285          0.000000     0.212    0.390   
2  False       0.0987      0.000455          0.000000     0.306    0.387   

     Tempo  
0  167.910  
1  185.081  
2  144.111  
Artist IDs            int32
Track Name           

In [30]:
# Масштабирование числовых признаков
scaler = StandardScaler()
numeric_features = df.select_dtypes(include=['int32', 'int64', 'float32', 'float64']).columns
df[numeric_features] = scaler.fit_transform(df[numeric_features])
print(df.head(3))

   Artist IDs  Track Name  Album Name  Artist Name(s)  Release Date  \
0   -1.032943   -1.047387   -1.132416       -1.214946     -1.191165   
1   -1.032943   -0.007295   -0.120908       -1.214946      0.549409   
2   -1.032943    1.330378    0.923455       -1.214946      1.555790   

   Duration (ms)  Popularity    Genres  Danceability    Energy       Key  \
0      -0.919780    2.410398  1.890198      0.252167  0.382949  0.498957   
1      -0.548958    2.298142  1.890198      0.317669  0.377046  1.066209   
2      -0.675508    2.522654  1.890198      0.536006  0.076001 -1.486428   

   Loudness   Mode  Speechiness  Acousticness  Instrumentalness  Liveness  \
0  1.034060   True     0.874084     -0.227497         -0.943709  2.487116   
1  0.992878  False     2.215229     -0.227993         -0.943718 -0.051936   
2  0.823040  False     0.090922     -0.226736         -0.943718  0.521792   

    Valence     Tempo  
0  1.007971  1.476925  
1  0.650282  2.060858  
2  0.634023  0.667594  


Разделение данных на обучающий и тестовый наборы

In [31]:
X = df.drop('Mode', axis=1)
Y = df['Mode']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

---
---

Поиск оптимальных параметров

In [32]:
svm_model = SVC()
parameters = {'kernel':('rbf', 'linear', 'sigmoid'), 'C':[0.1, 0.3, 1], 'gamma':[1, 10, 100]}
grid_search = GridSearchCV(svm_model, parameters, cv=3, verbose=3,n_jobs=1)
grid_search.fit(X_train, Y_train)

Fitting 3 folds for each of 27 candidates, totalling 81 fits
[CV 1/3] END ........C=0.1, gamma=1, kernel=rbf;, score=0.619 total time=  14.8s
[CV 2/3] END ........C=0.1, gamma=1, kernel=rbf;, score=0.619 total time=  13.1s
[CV 3/3] END ........C=0.1, gamma=1, kernel=rbf;, score=0.619 total time=  15.1s
[CV 1/3] END .....C=0.1, gamma=1, kernel=linear;, score=0.619 total time=   3.1s
[CV 2/3] END .....C=0.1, gamma=1, kernel=linear;, score=0.616 total time=  13.2s
[CV 3/3] END .....C=0.1, gamma=1, kernel=linear;, score=0.619 total time=   2.1s
[CV 1/3] END ....C=0.1, gamma=1, kernel=sigmoid;, score=0.533 total time=   4.3s
[CV 2/3] END ....C=0.1, gamma=1, kernel=sigmoid;, score=0.530 total time=   4.4s
[CV 3/3] END ....C=0.1, gamma=1, kernel=sigmoid;, score=0.523 total time=   5.7s
[CV 1/3] END .......C=0.1, gamma=10, kernel=rbf;, score=0.619 total time=   6.9s
[CV 2/3] END .......C=0.1, gamma=10, kernel=rbf;, score=0.619 total time=   7.7s
[CV 3/3] END .......C=0.1, gamma=10, kernel=rbf;

GridSearchCV(cv=3, estimator=SVC(), n_jobs=1,
             param_grid={'C': [0.1, 0.3, 1], 'gamma': [1, 10, 100],
                         'kernel': ('rbf', 'linear', 'sigmoid')},
             verbose=3)

Получение лучших параметров

In [33]:
best_parametrs = grid_search.best_params_
print(best_parametrs)

{'C': 1, 'gamma': 1, 'kernel': 'rbf'}


Обучение с лучшими параметрами

In [34]:
best_svm = grid_search.best_estimator_
best_svm.fit(X_train, Y_train)
# Обучение модели на тренировочных данных
svm_model.fit(X_train, Y_train)

SVC()

Предсказание на тестовом наборе

In [35]:
Y_pred = svm_model.predict(X_test)

Оценка производительности

In [37]:
accuracy = accuracy_score(Y_test, Y_pred)
result = classification_report(Y_test, Y_pred)

print(accuracy)
print(result)

0.6350308641975309
              precision    recall  f1-score   support

       False       0.60      0.14      0.22       991
        True       0.64      0.94      0.76      1601

    accuracy                           0.64      2592
   macro avg       0.62      0.54      0.49      2592
weighted avg       0.62      0.64      0.55      2592



Перекрестная проверка для оценки модели

In [40]:
cv_scores = cross_val_score(svm_model, X_train, Y_train, cv=3)

print(*cv_scores, sep=', ')
print(cv_scores.mean())

0.6390738060781476, 0.6364688856729378, 0.638205499276411
0.6379160636758322
